# Creator: Ivan Bardarov <br> (University of Strathclyde, March 2019)
## This module sets up the server and creates REST endpoints


In [3]:
from starlette.applications import Starlette
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles
import sys
sys.path.append('../recommender/')
from starlette.middleware.cors import CORSMiddleware
import uvicorn, aiohttp, asyncio
from io import BytesIO
from pathlib import Path
from cloth_detector import *
from style_recommender import *
import email_agent as ea

# application init
app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])

# expose the files in these folders to be allowed for acess
app.mount('/static', StaticFiles(directory='../static'))
app.mount('/static', StaticFiles(directory='../static/image_data'))
app.mount('/static', StaticFiles(directory='../static/tmp'))
app.mount('/static', StaticFiles(directory='../static/assets'))

# define JSON keys
ORIGINAL = "original_image"
CROPPED = "cropped_items"
CATS = "categories"
CHOSEN_IMAGE = "chosen_image"
CHOSEN_CAT = "chosen_cat"
ID2CAT = "id2cat"

@app.route('/')
def index(request):
    """
    An index API which returns the main webapge.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    HTMLResponse
        the requested html file

    """
    html = Path("../view/index.html")
    return HTMLResponse(html.open().read())


@app.route('/email', methods=['POST'])
async def send_email(request):
    """
    This function handles POST request sent to /email.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    JSONResponse
        Empty JSON response to finish handling the request

    """
    json = await request.json()
    #pass the request along
    await ea.send_email(json["name"], json["email"], json["comments"])
    print(json)
    return JSONResponse({})


@app.route('/analyze', methods=['POST'])
async def detect_items(request):
    """
    This function handles POST request sent to /analyze.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    JSONResponse
        a json object with the results array

    """
    request_json = await request.json()
    #pass the request along
    results_arr = get_recommendations(request_json[CHOSEN_IMAGE], request_json[CHOSEN_CAT])
    print(result)
    return JSONResponse({"results":results_arr})


@app.route('/detect', methods=['POST'])
async def analyze(request):
    """
    This function handles POST request sent to /detect.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    JSONResponse
        an array of result image paths

    """
    data = await request.form()
    #read the file
    img_bytes = await (data['file'].read())
    #pass the request along
    i_fns, cat_list, id2cat = get_detected_items(Image.open(BytesIO(img_bytes)))
    #construct the JSON response
    response = {CROPPED: i_fns,
                CATS: cat_list,
                ID2CAT: id2cat}
    print(response)
    return JSONResponse(response)


if __name__ == '__main__':
    uvicorn.run(app=app, host='0.0.0.0', port=1997)